In [1]:
#import libraries
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

C:\Users\Koushik\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Open the file and convert to lower characters
filename = 'alice.txt'
raw_text = open(filename).read()
raw_text = raw_text.lower()
print(raw_text[:1000])

alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, 'and what is the use of a book,' thought alice 'without pictures or
conversations?'

so she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure
of making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a white rabbit with pink eyes ran
close by her.

there was nothing so very remarkable in that; nor did alice think it so
very much out of the way to hear the rabbit say to itself, 'oh dear!
oh dear! i shall be late!' (when she thought it over afterwards, it
occurred to her that she ought to have wondered at this, but at the time
it all seemed quite natural); but when the rabbit actually took a watch
out of its waistcoat-pocket, and looked at it, and 

In [3]:
#For encoding and decoding we have to create a mapping dictionary from characters to integers and integer to characters
chars = sorted(list(set(raw_text)))
char_to_int = dict((c,i) for i,c in enumerate(chars))
int_to_char = dict((i,c) for i,c in enumerate(chars))

In [4]:
#print the summaries of the characters and vocabularies
no_of_chars = len(raw_text)
no_of_vocabulary = len(chars)
print("Total Characters: ", no_of_chars)
print("Total Vocabularies: ", no_of_vocabulary)
print(chars)
print(char_to_int)
print(int_to_char)

Total Characters:  144286
Total Vocabularies:  43
['\n', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', ':', ';', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, '*': 7, ',': 8, '-': 9, '.': 10, ':': 11, ';': 12, '?': 13, '[': 14, ']': 15, '_': 16, 'a': 17, 'b': 18, 'c': 19, 'd': 20, 'e': 21, 'f': 22, 'g': 23, 'h': 24, 'i': 25, 'j': 26, 'k': 27, 'l': 28, 'm': 29, 'n': 30, 'o': 31, 'p': 32, 'q': 33, 'r': 34, 's': 35, 't': 36, 'u': 37, 'v': 38, 'w': 39, 'x': 40, 'y': 41, 'z': 42}
{0: '\n', 1: ' ', 2: '!', 3: '"', 4: "'", 5: '(', 6: ')', 7: '*', 8: ',', 9: '-', 10: '.', 11: ':', 12: ';', 13: '?', 14: '[', 15: ']', 16: '_', 17: 'a', 18: 'b', 19: 'c', 20: 'd', 21: 'e', 22: 'f', 23: 'g', 24: 'h', 25: 'i', 26: 'j', 27: 'k', 28: 'l', 29: 'm', 30: 'n', 31: 'o', 32: 'p', 33: 'q', 34: 'r', 35: 's', 36: 't', 37: 'u', 38: 'v', 39: 'w', 40

In [5]:
#preparing the dataset for input and output pairs as encoders and decoders
seq_length = 100
X_data = []
y_data = []
for i in range(0,no_of_chars-seq_length,1):
  seq_in = raw_text[i:i+seq_length]
  seq_out = raw_text[i+seq_length]
  X_data.append([char_to_int[char] for char in seq_in])
  y_data.append(char_to_int[seq_out])
  

In [6]:
#print the number of patterns generated
no_patterns = len(X_data)
print("Number of patterns ",no_patterns)

Number of patterns  144186


In [7]:
#Reshape the X to be [samples,time steps,features]
X = (numpy.reshape(X_data,(no_patterns,seq_length,1)))
numpy.shape(X)

(144186, 100, 1)

In [8]:
#Normalize the data
X = (X /float(no_of_vocabulary))

In [9]:
# one hot encode the output variable
y = np_utils.to_categorical(y_data)

In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [ ]:
#here we define the checkpoint so that the trained model can be loaded later
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.text_gen2"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [12]:
model.fit(X,y,epochs=50,batch_size=128,callbacks=callbacks_list)

Epoch 1/50
144186/144186 [==============================] - 358s 2ms/step - loss: 1.5886

Epoch 00001: loss improved from inf to 1.58858, saving model to weights-improvement-01-1.5886.text_gen2
Epoch 2/50
144186/144186 [==============================] - 358s 2ms/step - loss: 1.5655

Epoch 00002: loss improved from 1.58858 to 1.56553, saving model to weights-improvement-02-1.5655.text_gen2
Epoch 3/50
144186/144186 [==============================] - 359s 2ms/step - loss: 1.5431

Epoch 00003: loss improved from 1.56553 to 1.54312, saving model to weights-improvement-03-1.5431.text_gen2
Epoch 4/50
144186/144186 [==============================] - 360s 2ms/step - loss: 1.5221

Epoch 00004: loss improved from 1.54312 to 1.52213, saving model to weights-improvement-04-1.5221.text_gen2
Epoch 5/50
144186/144186 [==============================] - 361s 3ms/step - loss: 1.5041

Epoch 00005: loss improved from 1.52213 to 1.50406, saving model to weights-improvement-05-1.5041.text_gen2
Epoch 6/50
144

In [10]:
import keras
new_model = keras.models.load_model('model.h5')
new_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 43)                11051     
Total params: 800,555
Trainable params:

In [25]:
# pick a random seed for generating the text
start = numpy.random.randint(0, len(X_data)-1)
pattern = X_data[start]
print ("Seed:")
print ("\"",''.join([int_to_char[value] for value in pattern]),"\"")

Seed:
" ly.

'yes,' said alice, 'we learned french and music.'

'and washing?' said the mock turtle.

'certa "


In [26]:
# generate characters
final = []
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x/float(no_of_vocabulary)
	prediction = new_model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
  #finale = ''.join(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

inly, i mnow that ' said alice, 'i mever keard the reason is it, i don't lnow that dorm that!'

'i mead the rueen?' said the mock turtle. 
'not at all come and all ratsing of mint,' said alice, 'i mean the bouldr then in the listle gresn thme, and the mosal of the toil it. the rueen's prete out of the way of the little goass to them the had not about the tea that she had not about the table, and she was a little book with the gale. 'the rueen of the tort of the thalp biange the tay that sald at the mock turtle, and the trees said to the gale.

'well, i suppose,' said alice, 'i mever keart there was the mock turtle so another, and the dreat hear of shehed to tea that she had been fornowsed to tee the had not about the thate.

'i mever heard of the louse, in and then seamed to tay the tay,' and she senoeed ierself ay the fuecutioner sabe wou cat a little without as the could, and see how suadenly was not a mittle wite all her hacd again, and the thouleed had tome thme the had not about t